# Bayesian Information Criterion (BIC)


In [13]:
import os
import pandas as pd
import glob
import math
import emcee
import string
import corner
import numpy as np
from tqdm import tqdm
from numba import jit
import concurrent.futures
from multiprocessing import Pool
from multiprocessing import cpu_count
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.ticker as mticker
from scipy.integrate import odeint
import pdse_project as pdse
import seaborn as sns
import scipy.stats as stats
import sys

data_files = sorted(glob.glob("./data/*.txt"))
full_data = {}

for file in data_files:
    data_type = os.path.basename(file).split('/')[-1].split('_c')[0]
    mouse_name = 'c' + file.split('/')[-1].split('_c')[-1].split('_t')[0]
    t_days = np.array([int(t) for t in file.split('/')[-1].split('_c')[-1].split('.txt')[0].split('_SUV')[0].split('_t')[1:]])
    suv_value = float(os.path.splitext(os.path.basename(file))[0].split("_SUV")[-1])
    data = np.loadtxt(file)
    treatment = os.path.basename(data_type).split('_')[0]

    if data_type not in full_data:
        full_data[data_type] = []

    full_data[data_type].append({
        'name': mouse_name,
        'data': data,
        'treatment': treatment,
        'treatment_days': t_days,
        'SUV': suv_value
    })

def calculate_n_total(full_data, group):
    return sum(len(mouse_data['data']) for mouse_data in full_data.get(group, []))

all_models = [
            '_ExponentialDecay',
            '_CumulativeDose',
            '_AccelED',
            '_AccelCD',
            '_MichaelisMenten',
            '_dePillisRadunskayaLaw',
            '_CDwithLogFactor',
            '_CDwithExpFactor',
            '_CDwithSinFactor',
            '_EDwithLogFactor',
            '_EDwithExpFactor',
            '_EDwithSinFactor', 
]

groups = ['io_sensitive']

group_ll_total = {g: {} for g in groups}
group_params_total = {g: {} for g in groups}
group_obs_total = {g: 0 for g in groups}
group_bic = {g: {} for g in groups}

for g in groups:
    n_total = calculate_n_total(full_data, g)
    group_obs_total[g] = n_total

    for model in all_models:
        files_location = f'./Output_Calibration/multi_ll_pars_{g}{model}.npz'
        if not os.path.exists(files_location):
            group_ll_total[g][model] = np.nan
            group_params_total[g][model] = 0
            group_bic[g][model] = np.nan
            print(f'[AVISO] Arquivo ausente: {files_location}')
            continue

        npzfile = np.load(files_location)
        max_ll = float(npzfile['max_ll'])
        theta = npzfile['pars']
        k = len(theta)

        group_ll_total[g][model] = max_ll
        group_params_total[g][model] = k

        if n_total > 0 and np.isfinite(max_ll):
            group_bic[g][model] = k * np.log(n_total) - 2.0 * max_ll
        else:
            group_bic[g][model] = np.nan

print("% --- BIC ---")
header = "group & " + " & ".join([m.replace('_', '') for m in all_models]) + " \\\\ \\hline"
print(header)

for g in groups:
    row = f"{g} "
    for model in all_models:
        bic_val = group_bic[g][model]
        row += "& --- " if not np.isfinite(bic_val) else f"& {bic_val:.2f} "
    print(row + "\\\\ \\hline")

print("\n% --- WBIC (BIC weights) ---")
header_w = "group & " + " & ".join([m.replace('_', '') for m in all_models]) + " \\\\ \\hline"
print(header_w)

for g in groups:
    bics = np.array([group_bic[g][m] for m in all_models], dtype=float)
    mask = np.isfinite(bics)

    if mask.sum() == 0:
        weights = np.full_like(bics, np.nan, dtype=float)
    else:
        bics_valid = bics[mask]
        delta = bics_valid - np.min(bics_valid)
        w_unnorm = np.exp(-0.5 * delta)
        w_norm = w_unnorm / np.sum(w_unnorm)

        weights = np.full_like(bics, np.nan, dtype=float)
        weights[mask] = w_norm

    row = f"{g} "
    for w in weights:
        row += "& --- " if not np.isfinite(w) else f"& {w:.3f} "
    print(row + "\\\\ \\hline")


% --- BIC ---
group & ExponentialDecay & CumulativeDose & AccelED & AccelCD & MichaelisMenten & dePillisRadunskayaLaw & CDwithLogFactor & CDwithExpFactor & CDwithSinFactor & EDwithLogFactor & EDwithExpFactor & EDwithSinFactor \\ \hline
io_sensitive & 1040.57 & 1092.54 & 879.45 & 1381.00 & 936.92 & 1063.83 & 1048.49 & 1449.18 & 998.29 & 1014.14 & 1031.95 & 1058.15 \\ \hline

% --- WBIC (BIC weights) ---
group & ExponentialDecay & CumulativeDose & AccelED & AccelCD & MichaelisMenten & dePillisRadunskayaLaw & CDwithLogFactor & CDwithExpFactor & CDwithSinFactor & EDwithLogFactor & EDwithExpFactor & EDwithSinFactor \\ \hline
io_sensitive & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\ \hline
